In [1]:
import os
import os.path as osp
from typing import Any, List, Tuple
import numpy as np
import pandas as pd

import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader


In [2]:
root = '../data'
dataset = 'cn15k'
entity_id = pd.read_csv(os.path.join(root, dataset, 'entity_id.csv'))
relation_id = pd.read_csv(os.path.join(root, dataset, 'relation_id.csv'))
triples = pd.read_csv(os.path.join(root, dataset, 'data.tsv'), sep='\t', header=None)
train_triples = pd.read_csv(os.path.join(root, dataset, 'train.tsv'), sep='\t', header=None)
val_triples = pd.read_csv(os.path.join(root, dataset, 'val.tsv'), sep='\t', header=None)
test_triples = pd.read_csv(os.path.join(root, dataset, 'test.tsv'), sep='\t', header=None)

In [18]:
entity_id.head()

,entity string,id
0,pricing,0
1,flow,1
2,paul,2
3,tantalus,3
4,turkey,4


In [19]:
relation_id.head()

,relation string,id
0,relatedto,0
1,hascontext,1
2,isa,2
3,synonym,3
4,atlocation,4


In [17]:
triples.head()

,0,1,2,3
0,0,0,2571,0.709293
1,156,0,2571,0.709293
2,258,0,2571,0.542790
3,306,0,2571,0.544275
4,356,0,2571,0.709293


In [6]:
len(triples)

241158

In [7]:
len(train_triples)+len(val_triples)+len(test_triples)

241158

In [11]:
train_triples.to_numpy()[0]

array([2.60000000e+02, 2.00000000e+00, 1.38950000e+04, 8.92708786e-01])

In [14]:
tph_array = np.zeros((len(relation_id), len(entity_id)))
hpt_array = np.zeros((len(relation_id), len(entity_id)))
for h_, r_, t_, w in triples.to_numpy():  # only training data
    h, r, t = int(h_), int(r_), int(t_)
    tph_array[r][h] += 1.
    hpt_array[r][t] += 1.
tph = np.mean(tph_array, axis=1)
hpt = np.mean(hpt_array, axis=1)

In [16]:
tph.shape, hpt.shape

((36,), (36,))

In [20]:
cons = []
index_cons = {}
# Load data into cons and index_cons
for _, row in entity_id.iterrows():
    # Add entity to cons list
    cons.append(row['entity string'])
    # Add entity and id mapping to index_cons dictionary
    index_cons[row['entity string']] = row['id']